# Homework Assignment #2
BAX 423 | Dane Hamlett

## Part I

### Create XML Schema

In [ ]:
<xs:schema attributeFormDefault="unqualified" elementFormDefault="qualified"
    xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="catalog">
    <xs:complexType>
      <xs:sequence>
        <xs:element name="book">
          <xs:complexType>
            <xs:sequence>
              <xs:element type="xs:string" name="author"/>
              <xs:element type="xs:string" name="title"/>
              <xs:element type="xs:string" name="genre"/>
              <xs:element type="xs:float" name="price"/>
              <xs:element type="xs:date" name="publish_date"/>
              <xs:element type="xs:string" name="description"/>
            </xs:sequence>
            <xs:attribute type="xs:string" name="id"/>
          </xs:complexType>
        </xs:element>
      </xs:sequence>
    </xs:complexType>
  </xs:element>
</xs:schema>

### Create XML File

In [49]:
xml_file = """
<catalog>
  <book id="bk102">
     <author>Ralls, Kim</author>
     <title>Midnight Rain</title>
     <genre>Fantasy</genre>
     <price>5.95</price>
     <publish_date>2000-12-16</publish_date>
     <description>A former architect battles corporate zombies, 
     an evil sorceress, and her own childhood to become queen 
     of the world.</description>
  </book>
</catalog>
"""

### Parse XML File and Retrieve Price

In [94]:
## Import Libraries
import xml.etree.ElementTree as et

## Parse XML
root = et.fromstring(xml_file)
for book in root.findall('book'):
    price = book.find('price').text
    print('Price: ' + price)

Price: 5.95


## Part II

### JSON Exercise

In [98]:
## Import Libraries
import urllib.request
import json
import pymssql

## Establish Connection to Local SQL Server Instance, Open Cursor, and Truncate Table
## IP Address and Password Obfuscated for Security Purposes
conn = pymssql.connect(host='XXX.X.XXX.XXX:2015', user='svc_hopper', password='XXXXXX', database='UCDavisDB')
cursor = conn.cursor()
cursor.execute('TRUNCATE TABLE [UCDavisDB].[dbo].[JSON_Table]')

## Establish Base API URL and an Empty List to House API Results
base_url = 'https://jsonplaceholder.typicode.com/posts'
results = []

## Confirm that the API Call Returns 100 Records
with urllib.request.urlopen(base_url) as url:
    data = json.loads(url.read().decode())
    length = len(data)
    print('Record Count = ' + str(length))

## Loop Through API Results, and Generate Rows to be Loaded into SQL Server
    for i,o in enumerate(data):
        userId = o['userId']
        id = o['id']
        title = o['title']
        body = o['body']
        row = userId, id, title, body
        results.append(row)

## Insert Records into SQL Server, and Commit the Command
cursor.executemany("INSERT INTO [UCDavisDB].[dbo].[JSON_Table] VALUES (%d, %d, %s, %s)", results)
conn.commit()

## Sample the Results
cursor.execute('SELECT TOP 1 * FROM [UCDavisDB].[dbo].[JSON_Table]')
for row in cursor:
    print('row = %r' % (row,))
    
## Confirm 100 Records Loaded
cursor.execute('SELECT COUNT(*) FROM [UCDavisDB].[dbo].[JSON_Table]')
for row in cursor:
    print('Record Count = %r' % (row))

## Close the Connection
conn.close()

Record Count = 100
row = (1, 1, 'sunt aut facere repellat provident occaecati excepturi optio reprehenderit', 'quia et suscipit\nsuscipit recusandae consequuntur expedita et cum\nreprehenderit molestiae ut ut quas totam\nnostrum rerum est autem sunt rem eveniet architecto')
Record Count = 100


### References

- API Reference: https://jsonplaceholder.typicode.com/
- PYMSSQL Python/SQL Server Library: http://pymssql.org/en/stable/pymssql_examples.html